# Kasen Kilonova Seed

In [ ]:
import numpy as np
# Code to inject a kilonova for the Kepler filter at different redshifts
def K2Kilonova(model,mag, Time, Start):
    Kepfilt = np.loadtxt('/Users/ryanr/Documents/PhD/coding/Filters/Kepler/Kepler.dat')
    path = '/Users/ryanr/Documents/PhD/coding/Kasen models/OrderedBNS/'
    files = ['t000A3.npy',
       't030A3p15.npy',
       't100A3p15.npy',
       't300A3p15.npy',
       'tinfA3p15.npy',
       't100A3p15_SD1e-2.npy',
       't100A3p15_SD1e-3.npy',
       't100A3p15_SD1e-4.npy',
       't100A3p15_SD1e-5.npy',
       'a80_leak_HR.npy']
    data = np.load(path+files[model]).item()
    time = data['time']
    # Make a 30 min interpolated model to fit more closely with Kepler frames.
    keptime = np.arange(time[0],time[-1],30*60)
    Ikep = np.interp(data['wavelength'], Kepfilt[:,0], Kepfilt[:,1])
    lum = data['luminosity']*Ikep
    Lum = np.zeros((lum.shape[0],lum.shape[1],1))
    Mag = np.zeros((lum.shape[0],lum.shape[1],1))
    Kc = np.zeros((lum.shape[0],lum.shape[1],1))
    kc = np.zeros((lum.shape[0],lum.shape[1],1))
    Kepcounts = np.zeros((lum.shape[0],keptime.shape[0]))
    KepMag = np.zeros((lum.shape[0],keptime.shape[0]))
    
    d = 10*3.1e18# 10 pc distance in pc to cm
    for i in range(lum.shape[0]):
        for j in range(lum.shape[1]):
            ind = ~np.isnan(lum[i,j,:])
            Lum[i,j] = np.trapz(lum[i,j,ind],data['wavelength'][ind])/3528.4*(5781.3**2/(3*10**18))/(4*np.pi*d**2)
            Mag[i,j] = -5/2*np.log10(Lum[i,j]) - 48.6
            
            # convert the counts from 10pc to those for a magnitude given to the function
            
        KepMag[i,:] = np.interp(keptime,time,Mag[i,:,0])
        
        Max = np.nanmin(KepMag[i,:])
        D = 10**((mag-Max+5)/5)
        Kepcounts[i,:] = 10**(-2/5*((KepMag[i,:] + 5*np.log10(D)-5)-25.47))
    Kepcounts = Kepcounts[0,keptime > 0]
    NS = np.zeros(len(Time))
    
    if len(Kepcounts) > len(Time[Start:]):
        NS[Start:] = Kepcounts[:(len(NS)-Start)]
    else:
        NS[Start:Start+len(Kepcounts)] = Kepcounts
    return NS